<a href="https://colab.research.google.com/github/ShuHuiK/WIE3007_Group_Assignment/blob/MinEn/Model_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. ENTER FINAL RESULTS

# Decision Tree
dt_results = {
    'Accuracy': 0.93,
    'Recall': 0.11,
    'Precision': 0.25,
    'F1-Score': 0.15
}

#neural network
nn_results = {
    'Accuracy': 0.89,
    'Recall': 0.50,
    'Precision': 0.28,
    'F1-Score': 0.36
}

# XgBoost
xgb_results = {
    'Accuracy': 0.90,
    'Recall': 0.07,
    'Precision': 0.29,
    'F1-Score': 0.11
}

# Random Forest
rf_results = {
    'Accuracy': 0.91,
    'Recall': 0.33,
    'Precision': 0.29,
    'F1-Score': 0.31
}

# KNN
knn_results = {
    'Accuracy': 0.80,
    'Recall': 0.44,
    'Precision': 0.14,
    'F1-Score': 0.21
}

# 2. BUILD THE LEADERBOARD
data = {
    'Model': ['Decision Tree (Baseline)', 'Neural Network', 'XGBoost', 'Random Forest', 'kNN'],
    'Accuracy': [dt_results['Accuracy'], nn_results['Accuracy'], xgb_results['Accuracy'], rf_results['Accuracy'], knn_results['Accuracy']],
    'Recall (Defaulters)': [dt_results['Recall'], nn_results['Recall'], xgb_results['Recall'], rf_results['Recall'], knn_results['Recall']],
    'Precision (Defaulters)': [dt_results['Precision'], nn_results['Precision'], xgb_results['Precision'], rf_results['Precision'], knn_results['Precision']],
    'F1-Score': [dt_results['F1-Score'], nn_results['F1-Score'], xgb_results['F1-Score'], rf_results['F1-Score'], knn_results['F1-Score']]
}

df_leaderboard = pd.DataFrame(data)

# Display Table
print("FINAL MODEL LEADERBOARD")
numeric_cols = ['Accuracy', 'Recall (Defaulters)', 'Precision (Defaulters)', 'F1-Score']
display(df_leaderboard.style.format("{:.2f}", subset=numeric_cols))


FINAL MODEL LEADERBOARD


,Model,Accuracy,Recall (Defaulters),Precision (Defaulters),F1-Score
0,Decision Tree (Baseline),0.93,0.11,0.25,0.15
1,Neural Network,0.89,0.50,0.28,0.36
2,XGBoost,0.90,0.07,0.29,0.11
3,Random Forest,0.91,0.33,0.29,0.31
4,kNN,0.80,0.44,0.14,0.21


In [2]:
# 1. Install required libraries
!pip install -q -U torch torchvision torchaudio transformers accelerate bitsandbytes

print("✅ Installation complete.")
print("⚠️ NOW: Go to the top menu, click 'Runtime' > 'Restart Session', then run Cell 2 below.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0

In [1]:
import torch
import os
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login

hf_token = os.getenv("HF_TOKEN")

try:
    if hf_token:
        login(token=hf_token)
        print("Logged in to Hugging Face successfully!")
    else:
        print("HF_TOKEN not found. Skipping Hugging Face login.")
except Exception as e:
    print(f"Login warning: {e}")

# LOAD MISTRAL 7B MODEL
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load in 4-bit mode (Required for free Colab T4 GPU)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

print("Loading Model... (This takes about 2 minutes)")
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config)
    print("Model Loaded!")
except Exception as e:
    print(f"Error Loading Model: {e}")
    print("TIP: Did you restart the runtime after installing bitsandbytes?")

# PREPARE THE DATA
# Leaderboard Data
data = {
    'Model': ['Decision Tree', 'Neural Network', 'XGBoost', 'Random Forest', 'kNN'],
    'Accuracy': [0.93, 0.89, 0.90, 0.91, 0.80],
    'Recall': [0.11, 0.50, 0.07, 0.33, 0.44],
    'F1-Score': [0.15, 0.36, 0.11, 0.31, 0.21]
}
df_leaderboard = pd.DataFrame(data)
leaderboard_txt = df_leaderboard.to_markdown(index=False)

# Feature Importance Data (From kNN Chart)
feature_txt = """
KNN PERMUTATION IMPORTANCE RESULTS:
(Positive value = Feature helps the model. Negative value = Feature confuses the model)
1. credit_score: +0.14 (Top Predictor)
2. debt_to_income: +0.08 (Strong Predictor)
3. savings_ratio: +0.03 (Weak Predictor)
4. income: -0.08 (Negative Impact)
5. payment_punctuality: -0.09 (Negative Impact)
6. credit_utilization: -0.11 (Most Harmful Feature)
"""

# GENERATE REPORT
prompt = f"""[INST]
You are a Lead Data Scientist writing a final project report.

Here is the Model Leaderboard:
{leaderboard_txt}

Here is the Feature Importance for the kNN model (Permutation Importance):
{feature_txt}

Please write the "Model Evaluation & Interpretation" section covering these 3 points:

1. **Summarise Findings:** Compare the models. Highlight that while Decision Tree has 93% Accuracy, the Neural Network is better because it has 50% Recall (catching more defaulters).
2. **Interpret Feature Importance:** Explain that 'credit_score' and 'debt_to_income' are the most important drivers of risk.
3. **Deep Insight (The Negative Values):** Observe that 'credit_utilization' has a NEGATIVE importance score (-0.11). Explain to the business that this feature might be noisy or confusing the kNN model, and we should consider removing it to improve performance.
[/INST]
"""

print("Generating Analysis...\n")
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=600)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print Final Output
print(response)

Logged in to Hugging Face successfully!
Loading Model... (This takes about 2 minutes)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model Loaded!
Generating Analysis...

[INST]
You are a Lead Data Scientist writing a final project report.

Here is the Model Leaderboard:
| Model          |   Accuracy |   Recall |   F1-Score |
|:---------------|-----------:|---------:|-----------:|
| Decision Tree  |       0.93 |     0.11 |       0.15 |
| Neural Network |       0.89 |     0.5  |       0.36 |
| XGBoost        |       0.9  |     0.07 |       0.11 |
| Random Forest  |       0.91 |     0.33 |       0.31 |
| kNN            |       0.8  |     0.44 |       0.21 |

Here is the Feature Importance for the kNN model (Permutation Importance):

KNN PERMUTATION IMPORTANCE RESULTS:
(Positive value = Feature helps the model. Negative value = Feature confuses the model)
1. credit_score: +0.14 (Top Predictor)
2. debt_to_income: +0.08 (Strong Predictor)
3. savings_ratio: +0.03 (Weak Predictor)
4. income: -0.08 (Negative Impact)
5. payment_punctuality: -0.09 (Negative Impact)
6. credit_utilization: -0.11 (Most Harmful Feature)


Please 